In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_selection import SelectPercentile

from sklearn.datasets import load_breast_cancer


In [21]:
#Make CSV like data frame
data = load_breast_cancer()
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['target'] = data['target']


df = pd.read_csv('/Volumes/researchFiles/test_files/large_file.csv')
missing = df[df.isna().any(axis=1)]
df = df.drop(missing.index)
target = df.columns[-1]

In [22]:


#Make training and testing data
X = df.drop(columns=[target])
y = df[target]
feature_names = X.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5, stratify=y)


#Select Percentile
feature_scores = f_classif(X_train, y_train)[0]
output_select_percentile = []

for score, f_name in sorted(zip(feature_scores, feature_names), reverse=True):

    output_select_percentile.append({
        'feature': f_name,
        'score': score
    })

#Random Forrest
forest = RandomForestClassifier(random_state=5)
rf_best = forest.fit(X_train, y_train)
output_rf = []

for score, f_name in sorted(zip(rf_best.feature_importances_, feature_names), reverse=True):
    output_rf.append({
        'feature': f_name,
        'score': score
    })


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [23]:
percentile_array = [75, 50, 25]


#Manual calculation of percentile
def calculate_percentile_column_count(data_array, percentile):
  length = len(data_array)
  last_column = round(length * percentile/100)
  return len(data_array[0:last_column])

In [24]:
#compare manual calculation of percentile to sklearn build in select percentile
result = []
for percentile in percentile_array:
  result.append({
      'method': 'manual',
      'percentile': percentile,
      'type': 'select_percentile',
      'count': calculate_percentile_column_count(output_select_percentile, percentile)
  })
  result.append({
      'method': 'manual',
      'percentile': percentile,
      'type': 'rf',
      'count': calculate_percentile_column_count(output_rf, percentile)
  })

  model = SelectPercentile(percentile=percentile)
  X_new = model.fit_transform(X_train, y_train)
  result.append({
      'method': 'skl_select_percentile',
      'percentile': percentile,
      'type': 'select_percentile',
      'count': len(X_train.columns[model.get_support()])
  })

In [25]:
pd.DataFrame(result)

,method,percentile,type,count
0,manual,75,select_percentile,23
1,manual,75,rf,23
2,skl_select_percentile,75,select_percentile,23
3,manual,50,select_percentile,16
4,manual,50,rf,16
5,skl_select_percentile,50,select_percentile,15
6,manual,25,select_percentile,8
7,manual,25,rf,8
8,skl_select_percentile,25,select_percentile,8
